# Exoplanet Habitability Analysis - A Machine Learning EDA

##### A quick note: I've always loved outer space since I was a child, fascinated by planet, asteroids, stars, galaxies, and other celestial objects. From reading books explaining space, to watching theories on TV and movies, it is something I never really grew out of. Now as my final year in my BS program comes to a close, I'm excited to combine my passions for research and outer space in a solo endeavor. 

### TABLE OF CONTENTS:
- TBA

### Flow: This EDA will analyze the exoplanets within the NASA Exoplanet Archive as of Sunday, October 5th, 2025. This analysis will classify exoplanets in 2 ways:
- 1. VIA XGBoost (great for handling missing values)

In [6]:
# imports 
# Standard library
import time

# Core libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, roc_auc_score

# PyTorch
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

# Scikit-learn - model selection
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV
)

# Scikit-learn - preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    LabelEncoder,
    PolynomialFeatures
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Scikit-learn - models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier,
    AdaBoostClassifier
)
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC

# Scikit-learn - feature selection and calibration
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.calibration import CalibratedClassifierCV

# Scikit-learn - evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
    roc_curve,
    auc,
    average_precision_score
)

# Imbalanced-learn
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# Scipy
from scipy.stats import loguniform


# load csv into a pandas DataFrame
exoplanet_df = pd.read_csv("data/exoplanet_data.csv", comment = "#")



In [7]:
# Data preprocessing and cleaning
# PART 1: Prepping for classification


# there are around ~6000 exoplanets currently discovered, but there are many rows due to multiple discoveries
# must remove these extras so we only have 1 row/planet. Setting default flag to 1 to get most accurate/accepted data on planet
exoplanet_df = exoplanet_df[exoplanet_df["default_flag"] == 1]
print(exoplanet_df.info())
print(exoplanet_df.describe())



<class 'pandas.core.frame.DataFrame'>
Index: 6028 entries, 0 to 38971
Data columns (total 92 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pl_name          6028 non-null   object 
 1   hostname         6028 non-null   object 
 2   default_flag     6028 non-null   int64  
 3   sy_snum          6028 non-null   int64  
 4   sy_pnum          6028 non-null   int64  
 5   discoverymethod  6028 non-null   object 
 6   disc_year        6028 non-null   int64  
 7   disc_facility    6028 non-null   object 
 8   soltype          6028 non-null   object 
 9   pl_controv_flag  6028 non-null   int64  
 10  pl_refname       6028 non-null   object 
 11  pl_orbper        5709 non-null   float64
 12  pl_orbpererr1    5220 non-null   float64
 13  pl_orbpererr2    5220 non-null   float64
 14  pl_orbperlim     5709 non-null   float64
 15  pl_orbsmax       3757 non-null   float64
 16  pl_orbsmaxerr1   2890 non-null   float64
 17  pl_orbsmaxerr2   2

In [8]:
# let's see what the first 10 rows look like just to get a feel for the data
print(exoplanet_df.head(10))

                    pl_name               hostname  default_flag  sy_snum  \
0                  11 Com b                 11 Com             1        2   
5                  11 UMi b                 11 UMi             1        1   
7                  14 And b                 14 And             1        1   
9                  14 Her b                 14 Her             1        1   
17               16 Cyg B b               16 Cyg B             1        3   
23                 17 Sco b                 17 Sco             1        1   
25                 18 Del b                 18 Del             1        2   
29  1RXS J160929.1-210524 b  1RXS J160929.1-210524             1        1   
31                 24 Boo b                 24 Boo             1        1   
33                 24 Sex b                 24 Sex             1        1   

    sy_pnum  discoverymethod  disc_year  \
0         1  Radial Velocity       2007   
5         1  Radial Velocity       2009   
7         1  Radial Vel

### First, we need a target. There is no "habitable" column, so we'll create one by defining certain parameters that are similar to those of a habitable planet 

In [9]:
# habitable planet = rocky surface, temperate, single star system 
# rough definition, but just broad so we have something to go off of

radius_cond = (exoplanet_df["pl_rade"] >= 0.5) & (exoplanet_df["pl_rade"] <= 1.5)
temp_cond = (exoplanet_df["st_teff"] >= 2600) & (exoplanet_df["st_teff"] <= 10000)
starNum_cond = (exoplanet_df["sy_snum"] == 1) 
flux_cond = (exoplanet_df["pl_insol"] >= 0.3) & (exoplanet_df["pl_insol"] <= 1.8)

exoplanet_df["potentially_habitable"] = (radius_cond & temp_cond & starNum_cond & flux_cond).fillna(False)
# the column looks like this
#print(exoplanet_df["potentially_habitable"] == True)

# see how many candidates we have just from the initial cate
print(exoplanet_df["potentially_habitable"].value_counts())

names = exoplanet_df.loc[exoplanet_df["potentially_habitable"] == True, "pl_name"]
for name in names:
    print(name)


potentially_habitable
False    6018
True       10
Name: count, dtype: int64
Gliese 12 b
K2-3 d
K2-72 e
Kepler-1649 c
Kepler-186 f
Kepler-438 b
Kepler-442 b
LP 890-9 c
TOI-700 d
TOI-700 e


### Now, we have our "habitable planets". As expected, we have way more false, than true. This means we have an unbalanced dataset.

#### I think we can find more. The main issue is the column "pl_insol". This is the insolation flux, an **extremely** important factor in planet habitability.
#### Unfortunately, this column has a high missing value 

In [10]:
print(exoplanet_df["pl_insol"].isnull().sum())

5143


#### 5143 out of 6022 entries have a missing pl_insol value, and with that being one of my main categories for habitability, it needs to be worked with


### DATA PREPROCESSING
- dropping irrelevant/redundant columns
- handling missing vals
- standardize formatting
- scaling features for model training

In [22]:
# STEP 1: handpick the most infuential columns for determining habitability 
# and make a new dataframe

new_cols = ["pl_name", "sy_snum", "sy_pnum", "pl_rade", "pl_orbeccen", 
            "pl_insol", "pl_eqt", "st_teff", 
            "st_rad", "st_logg", "potentially_habitable"]
df = exoplanet_df[new_cols].copy() 
df = df.drop(columns=["pl_name"])

# show new info
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
Index: 6028 entries, 0 to 38971
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sy_snum                6028 non-null   int64  
 1   sy_pnum                6028 non-null   int64  
 2   pl_rade                4499 non-null   float64
 3   pl_orbeccen            2546 non-null   float64
 4   pl_insol               885 non-null    float64
 5   pl_eqt                 1627 non-null   float64
 6   st_teff                5338 non-null   float64
 7   st_rad                 5257 non-null   float64
 8   st_logg                4995 non-null   float64
 9   potentially_habitable  6028 non-null   bool   
dtypes: bool(1), float64(7), int64(2)
memory usage: 476.8 KB
None
          sy_snum      sy_pnum      pl_rade  pl_orbeccen      pl_insol  \
count  6028.00000  6028.000000  4499.000000  2546.000000    885.000000   
mean      1.10418     1.768414     4.428962     0.150655   

In [28]:
# Training/splitting

X = df.drop(columns = ["potentially_habitable"])
y = df["potentially_habitable"].astype(int) # target, make it an int
# most functions assume positive (true) case = 1, so we make the change that true = 1, false = 0
# keep track of missing vals for pl_insol
X["pl_insol_missing"] = X["pl_insol"].isna().astype(int)

# reduce overfitting by using early stopping once new gains aren't effective anymore
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=42)

# class weight calc
pos = int(y_train.sum())
neg = int(len(y_train) - pos)
scale_pos_weight = neg / pos if pos > 0 else 1.0


print(f"Shapes: {X_train.shape} {X_val.shape} {X_test.shape}")
print(f"Train pos/neg: {pos } / {neg} => scale_pos_weight = {round(scale_pos_weight, 2)}")
print(f"Val pos rate: {round(y_val.mean(), 4)} Test pos rate: {round(y_test.mean(), 4)}")




Shapes: (3616, 10) (1206, 10) (1206, 10)
Train pos/neg: 6 / 3610 => scale_pos_weight = 601.67
Val pos rate: 0.0017 Test pos rate: 0.0017


#### Results Meaning
- There are only 6 positives to learn from
- Due to the splits, there will only be 2 positives in val and test
- Basically, there's too little actual habitble cases based on my requirements to draw any meaningful conclusions
- This make sense, if it were that easy to find another earth-like exolpanet, we'd probably be looking into things a lot heavier as a society.
- I'll reframe my approach to a rank-like system instead of a pure "yes or no". This way, the analysis can guage and talk about what makes a plane more habitable than another, rather than "this is habitable and this is not"
- soln: keep the binary classification approach w/ xgboost, using the probability score as a pseudo ranking

In [29]:
from sklearn.model_selection import RepeatedStratifiedKFold

top_k = [10, 25, 50, 100]

# wanna return recall, precision, and lift for each k
def topk_stats(y_true, scores, ks=top_k):
    order = np.argsort(-scores)
    
    y_sorted = np.asarray(y_true)[order]
    P = y_sorted.sum()
    base_rate = P / len(y_sorted) if len(y_sorted) else 0.0

    out = {}
    for k in ks:
        k = min(k, len(y_sorted))
        hits = y_sorted[:k].sum()
        recall = hits / P if P > 0 else 0.0
        precision = hits / k if k > 0 else 0.0
        lift = (precision / base_rate) if base_rate > 0 else 0.0
        out[k] = {"recall": recall, "precision": precision, "lift": lift}
    return out

rskf = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=42)


cv_rows = []
for fold_idx, (tr, va) in enumerate(rskf.split(X, y), start=1):
    X_tr, X_va = X.iloc[tr], X.iloc[va]
    y_tr, y_va = y.iloc[tr], y.iloc[va]

    pos = int(y_tr.sum())
    neg = int(len(y_tr) - pos)
    spw = (neg / pos) if pos > 0 else 1.0

# xgboost implementattion w/ early stopping

    clf = XGBClassifier(
        objective = "binary:logistic",
        tree_method = "hist",
        n_estimators=2000,
        max_depth=4,
        min_child_weight=30,
        subsample=0.8,
        colsample_bytree=0.8,
        learning_rate=0.08,
        reg_alpha=0.0,
        reg_lambda=5.0,
        scale_pos_weight=spw,
        n_jobs=-1,
        eval_metric="aucpr",
        random_state=fold_idx,
        verbosity=0,
        early_stopping_rounds=50,
    )

    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False
    )


    # probability as ranking score
    scores = clf.predict_proba(X_va)[:, 1]
    stats = topk_stats(y_va.values, scores, ks=top_k)

    for k, d in stats.items():
        cv_rows.append({
            "fold": fold_idx, "k": k,
            "recall": d["recall"], "precision": d["precision"], "lift": d["lift"],
            "best_iter": getattr(clf, "best_iteration", None)
        })

cv_df = pd.DataFrame(cv_rows)
summary = (cv_df.groupby("k")[["recall","precision","lift"]]
           .agg(["mean","std"]).reset_index())
summary

k    recall           precision                  lift           
            mean       std      mean       std        mean        std
0   10  0.861111  0.166667  0.288889  0.078174  173.027778  33.497946
1   25  0.962963  0.111111  0.128889  0.026667   77.397037   8.933910
2   50  1.000000  0.000000  0.066667  0.010000   40.186667   0.010000
3  100  1.000000  0.000000  0.033333  0.005000   20.093333   0.005000

In [ ]:
best_iters = [b for b in cv_df["best_iter"].dropna().tolist() if b is not None]
best_n = int(np.median(best_iters)) if best_iters else 300  # fallback

pos_all = int(y.sum()); neg_all = len(y) - pos_all
spw_all = (neg_all / pos_all) if pos_all > 0 else 1.0

final_clf = XGBClassifier(
    objective="binary:logistic",
    tree_method="hist",
    n_estimators=best_n,    
    max_depth=4,
    min_child_weight=30,
    subsample=0.8,
    colsample_bytree=0.8,
    learning_rate=0.08,
    reg_alpha=0.0,
    reg_lambda=5.0,
    scale_pos_weight=spw_all,
    n_jobs=-1,
    eval_metric="aucpr",
    random_state=123,
    verbosity=0,
)

final_clf.fit(X, y)  # no early stopping on the full fit


scores = final_clf.predict_proba(X)[:, 1]
rank_df = (exoplanet_df[["pl_name","pl_rade","pl_insol","pl_eqt","st_teff","st_rad","pl_orbeccen"]]
           .assign(score=scores, pl_insol_missing=X["pl_insol_missing"].values)
           .sort_values("score", ascending=False)
           .reset_index(drop=True))

rank_df.head(50)   


,pl_name,pl_rade,pl_insol,pl_eqt,st_teff,st_rad,pl_orbeccen,score,pl_insol_missing
0,TOI-700 d,1.073000,0.850,NaN,3459.00,0.421000,0.042,0.974724,0
1,TOI-700 e,0.953000,1.270,NaN,3459.00,0.421000,0.059,0.974724,0
2,Kepler-1649 c,1.060000,0.750,234.0,3240.00,0.231700,NaN,0.969652,0
3,Kepler-438 b,1.120000,1.400,NaN,3748.00,0.520000,0.030,0.968688,0
4,Kepler-186 f,1.170000,0.300,NaN,3755.00,0.523000,0.040,0.967545,0
5,Kepler-442 b,1.340000,0.660,NaN,4402.00,0.598000,0.040,0.964660,0
6,TRAPPIST-1 d,0.788000,1.115,NaN,2566.00,0.119200,NaN,0.960709,0
7,TRAPPIST-1 e,0.920000,0.646,NaN,2566.00,0.119200,NaN,0.957862,0
8,LP 890-9 c,1.367000,0.906,272.0,2850.00,0.155600,NaN,0.956594,0
9,K2-72 e,1.290000,1.200,NaN,3360.47,0.330989,0.110,0.951994,0
